In [1]:
pip install chess

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pygame
import chess
import math
import random
import chess.polyglot
import torch
import time
import copy

pygame 2.5.0 (SDL 2.28.0, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#initialise display
X = 800
Y = 800
scrn = pygame.display.set_mode((X, Y))
pygame.init()
reader = chess.polyglot.open_reader('baron30.bin')
#basic colours
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)


piece_width = 100
piece_height = 100

# Load and resize the piece images
pieces = {
    'p': pygame.transform.scale(pygame.image.load('images/b_p.png').convert(), (piece_width, piece_height)),
    'n': pygame.transform.scale(pygame.image.load('images/b_n.png').convert(), (piece_width, piece_height)),
    'b': pygame.transform.scale(pygame.image.load('images/b_b.png').convert(), (piece_width, piece_height)),
    'r': pygame.transform.scale(pygame.image.load('images/b_r.png').convert(), (piece_width, piece_height)),
    'q': pygame.transform.scale(pygame.image.load('images/b_q.png').convert(), (piece_width, piece_height)),
    'k': pygame.transform.scale(pygame.image.load('images/b_k.png').convert(), (piece_width, piece_height)),
    'P': pygame.transform.scale(pygame.image.load('images/w_p.png').convert(), (piece_width, piece_height)),
    'N': pygame.transform.scale(pygame.image.load('images/w_n.png').convert(), (piece_width, piece_height)),
    'B': pygame.transform.scale(pygame.image.load('images/w_b.png').convert(), (piece_width, piece_height)),
    'R': pygame.transform.scale(pygame.image.load('images/w_r.png').convert(), (piece_width, piece_height)),
    'Q': pygame.transform.scale(pygame.image.load('images/w_q.png').convert(), (piece_width, piece_height)),
    'K': pygame.transform.scale(pygame.image.load('images/w_k.png').convert(), (piece_width, piece_height)),
}

def update(scrn,board):
    '''
    updates the screen basis the board class
    '''
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece == None:
            pass
        else:
            scrn.blit(pieces[str(piece)],((i%8)*100,700-(i//8)*100))
    
    for i in range(7):
        i=i+1
        pygame.draw.line(scrn,WHITE,(0,i*100),(800,i*100))
        pygame.draw.line(scrn,WHITE,(i*100,0),(i*100,800))

    pygame.display.flip()


In [5]:
def main(BOARD):
    scrn.fill(BLACK)
    pygame.display.set_caption('Chess')
    index_moves = []
    status = True
    while (status):
        update(scrn,BOARD)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                status = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                scrn.fill(BLACK)
                pos = pygame.mouse.get_pos()
                square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                index = (7-square[1])*8+(square[0])
                if index in index_moves: 
                    
                    move = moves[index_moves.index(index)]
                    
                    BOARD.push(move)
                    index=None
                    index_moves = []
                else:
                    #check the square that is clicked
                    piece = BOARD.piece_at(index)
                    #if empty pass
                    if piece == None:
                        
                        pass
                    else:
                        all_moves = list(BOARD.legal_moves)
                        moves = []
                        for m in all_moves:
                            if m.from_square == index:                
                                moves.append(m)
                                t = m.to_square
                                TX1 = 100*(t%8)
                                TY1 = 100*(7-t//8)
                                pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                        
                        index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

def main_one_agent(BOARD,agent,agent_color):
    scrn.fill(BLACK)
    #name window
    pygame.display.set_caption('Chess')
    index_moves = []
    l = []
    status = True
    while (status):
        update(scrn,BOARD)  
        if BOARD.turn==agent_color:
            start_time = time.time()
            tree = [[[BOARD, [0], eval_board(BOARD), 0, 0, 0]]]
            move, l = agent(tree,l)
            BOARD.push(move)
            scrn.fill(BLACK)
            print("time used:", time.time()-start_time, "seconds")
        else:

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    status = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    scrn.fill(BLACK)
                    pos = pygame.mouse.get_pos()
                    square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                    index = (7-square[1])*8+(square[0])
                    if index in index_moves:     
                        move = moves[index_moves.index(index)]
                        #print(BOARD)
                        #print(move)
                        BOARD.push(move)
                        index=None
                        index_moves = []
                    else:                   
                        piece = BOARD.piece_at(index)                    
                        if piece == None:
                            pass
                        else:

                            all_moves = list(BOARD.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:          
                                    moves.append(m)
                                    t = m.to_square
                                    TX1 = 100*(t%8)
                                    TY1 = 100*(7-t//8)
                                    pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                            #print(moves)
                            index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

In [6]:
#game state analysis
def pseudoStep(a,b,c,e,x):
    return(a+(b-a)*(np.arctan((x-c)*e)/np.pi + 1/2)) # The higher is e, the greater is the slop
def smooth_indicator(x, a, b):
    e = 10/(b-a)
    left = pseudoStep(0,1,a, e,x)
    right = pseudoStep(1,0,b, e,x)
    return (left+right-1)*100
def total_value(board):
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    total = 0
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        piece_value = values[piece_type-1]
        total += piece_value
    return(total)
def game_stage(value):
    #100 =>0
    #mid game tipping point : 90
    #end game tipping point : 60
    #king_pawn tipping point : 42
    # prediction : probability of being at opening, mid, end, king-pawn
    prediction = np.array([smooth_indicator(value,90, 100),smooth_indicator(value,60, 90),smooth_indicator(value,42, 60),smooth_indicator(value,0, 42)])
    norm = np.sum(prediction)
    prediction = 100*prediction/norm
    prediction = [int(p) / 100 for p in prediction]
    return(prediction)

In [7]:
#position analysis
def is_passing_pawn(board, square, color):
    rank = chess.square_rank(square)
    file = chess.square_file(square)
    if file == 0:
         n_files = [0,1]
    if file == 7:
        n_files = [6,7]
    else:
        n_files = [file-1, file, file+1]
    if (color == True):
        for f in n_files:
            for r in range(rank+1, 8):
                square = rank * 8 + file
                if board.piece_at(square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)
    else:
        for f in n_files:
            for r in range(0, rank):
                square = rank * 8 + file
                if board.piece_at(square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)

    return False
def pawn_value(n, prediction, passing):
    value = np.array([[1,1,1,1.05,0.95,0.7,1.4,6], [1,1,1,1.05,1,0.8,1.5,6], [1,1,1.1,1.1,1.2,1.3,1.8,6], [1,1,1.2,1.4,1.6,1.9,2.7,6]])
    passing_value = np.array([[1,1,1,1,0.95,1.2,1.4,6], [1,1,1,1.05,1,0.8,1.5,6], [1,1,1.2,1.3,1.4,1.6,1.8,6], [1,1,1.2,1.5,1.8,2.2,2.7,6]])
    if passing:
        return np.sum(passing_value[:,n]*prediction)
    else:
        return np.sum(value[:,n]*prediction)
def king_value(file, n, prediction):
    file_value = np.array([[0.2,0.8,0.7,0,0,0.7,0.8,0.2],[0.2,0.9,0.8,0,0,0.8,0.9,0.2],[0,0.3,0.5,0.6,0.6,0.5,0.3,0],[0,0.3,0.5,0.5,0.5,0.5,0.3,0]])
    rank_value = np.array([[2, 1.7, 1.4, 1, 0.6, 0, 0,0],[2, 1.8, 1.5, 1.1, 0.6, 0, 0,0],[1.1, 1.3, 1.5, 1.7, 1.8, 1.6, 1.4, 1.2],[0.7, 1, 1.4, 1.7, 1.9, 2, 1.2,0.7]])
    return np.sum(file_value[:,file]*prediction + rank_value[:,n]*prediction)


In [8]:
#eval
def eval_board(board):
    #will be replaced by a deep network
    #tensor = board_to_tensor(board)[0]
    score = 0
    if board.is_checkmate() and board.turn:
        return(-1000)
    if board.is_checkmate() and board.turn == False:
        return(1000)
    if board.is_stalemate():
        return(0)
    if board.is_insufficient_material():
        return(0)
    if board.has_insufficient_material(board.turn):
        return(-100)
    if board.has_insufficient_material(board.turn == False):
        return(100)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, is_passing_pawn(board, square, piece.color))* color
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += king_value(chess.square_file(square), n, prediction) * color
        score += piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            dist = ((chess.square_rank(s) - 3.5)**2 + abs(chess.square_file(s) - 3.5)**2)
            score += 0.05*color*(prediction[0]/0.61/dist**0.33+prediction[1]/0.69/dist**0.25 + prediction[0]+prediction[1])
    turn = (1 if board.turn else -1)
    score += (0.25 if board.has_kingside_castling_rights(True) else 0)
    score += (-0.25 if board.has_kingside_castling_rights(False) else 0)
    score += (0.25 if board.has_queenside_castling_rights(True) else 0)
    score += (-0.25 if board.has_queenside_castling_rights(False) else 0)
    ajusted = ajust_score(board, int(score*100)/100)
    return ajusted

def stable_eval(b, n):
    value = eval_board(b)
    best = value
    m = 0
    B = b.copy()
    if n<=0:
        return(value)
    moves = list(b.legal_moves)
    for move in moves:
        if b.is_capture(move):
            copy = b.copy()
            copy.push(move)
            v = stable_eval(copy, n-1)
            gain = (v-value)*(1 if b.turn else -1)
            if (gain>1) and (gain > m) :
                m = gain
                best = v
    return(best)

def calculate_piece_sum(board):
    values = [1, 3, 3, 5, 9, 0]

    white_sum = 0
    black_sum = 0

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            if piece.color == chess.WHITE:
                white_sum += values[piece.piece_type-1]
            else:
                black_sum += values[piece.piece_type-1]

    return white_sum, black_sum
def ajust_score(board, score):
    white, black = calculate_piece_sum(board)
    if (score >0) and (white>black):
        return(score*white/black)
    elif (score <0) and (white<black):
        return(score*black/white)
    else:
        return(score)

In [9]:
#deep eval
def board_to_tensor(board):
    tensor = np.zeros((1,15, 8, 8), dtype=np.float32)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        layer = int(piece_type-1 + 3*(1+color))
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, is_passing_pawn(board, square, piece.color))
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += king_value(chess.square_file(square), n, prediction)
        tensor[0,layer, chess.square_rank(square), chess.square_file(square)] = piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            tensor[0,layer, chess.square_rank(s), chess.square_file(s)] += 0.05*color
    turn = (1 if board.turn else -1)
    tensor[0, 12, 0, 0] = (0.25 if board.has_kingside_castling_rights(True) else 0)
    tensor[0, 12, -1, 0] = (-0.25 if board.has_kingside_castling_rights(False) else 0)
    tensor[0, 12, 0, -1] = (0.25 if board.has_queenside_castling_rights(True) else 0)
    tensor[0, 12, -1, -1] = (-0.25 if board.has_queenside_castling_rights(False) else 0)
    tensor[0, 13, :, :] = turn/640
    if board.is_checkmate():
        tensor[0, 14, :, :] = -turn
    return torch.from_numpy(tensor)

def fen_to_tensor(fen):
    board = chess.Board(fen)
    return(board_to_tensor(board))

In [10]:
#tree[layer,...]
#layer[nodes,...]
#initial tree = [[[chess.Board(), [0], value, 0 ,width[1], width[2]/a, 0]]]
#node [board, origin, value, up_propagation_value, fertility, children fertility, recycle]
def darwinian_tree(tree, l, parameters, eval_board, color):
    depth = parameters.depth
    lib = []
    moves = list(b.legal_moves)
    if len(moves) ==0:
        return(chess.Move.null(), [[]])
    else :
        all_moves = list(reader.find_all(b))
        
        if all_moves == []:
            pass
        else:
            random.shuffle(all_moves)
            opening_move = all_moves[0]
            return (opening_move.move, [])
        rec, move = recycle(l, tree[-1][0][0], color)
        if rec:
            print("recycle")
            return(move, [])
        else:
            capture, tree = init(tree, eval_board, parameters, color)
            if capture:
                return(tree[0].pop(), [])
            #now tree_depth = 3
            print("start exploration")
            for i in range(2,depth):       
                side = ((i%2 == 0)== color)  #if color == False, i ==0 : side = False
                print("reproducing")
                children = reproduce(tree[-1], eval_board,False, side)
                print("growing")
                hunters, dict_ = growing(children, parameters, eval_board, side)
                print("hunting")
                hunters = hunt(hunters, len(tree), parameters, side)
                hunters = reasign(hunters, dict_)
                print("selecting")
                survived = select(hunters, side, eval_board, i, parameters)
                tree.append(survived)
                print("exploration " + str((i+1)*100/depth) + " %")
            length = []
            for t in tree:
                length.append(len(t))
            print(length)
            print("up propagating")
            for k in range(len(tree[-1])):
                tree[-1][k][3] = tree[-1][k][2]
            best_move, l = up_propagate(tree.copy(), color==(depth%2==1), [])
            print("showing steps")
            print_steps(l)
            length = []
            for t in tree:
                length.append(len(t))
            print(length)
            return(best_move, l[1:])
            
            
            

In [11]:
def init(tree, eval_board, parameters, side):
    print("-------------")
    print("initial")
    print("initial board value = " + str(eval_board(b, 0)))
    print("game_stage = " + str(game_stage(total_value(b))))
    print(b)
    temp = reproduce(tree[-1], eval_board, True, side)
    if len(temp[0][0])==1:
        return(True, temp[0][0][0])
    children = []
    for t in temp:
        children += t[0]
    tree.append(children)
    print("1st")
    print("width[1] = " + str(len(children)))
    temp = reproduce(tree[-1], eval_board, False, side==False)
    
    children = []
    for t in temp:
        s, k = keep(t[0], int(parameters.width[2]/len(tree[-1])), eval_board, 1, parameters, side==False)
        children += s
    tree.append(children)
    print("width[2] = " + str(len(children)))
    return(False, tree)

In [12]:
#reproduction
def reproduce(parents, eval_board, init, side):
    children = []
    for k in range(len(parents)):
        #if k%10==0:
            #print("reproduction : " + str(k/len(parents)*100) + " %")
        b = parents[k][0]
        parent_value = parents[k][2]
        moves = list(b.legal_moves)
        temp = []
        if len(moves)==0:
            value = eval_board(b, 0)
            temp = [[b.copy(), parents[k][1]+ [-1], value, 0 ,0]]
            children.append([temp.copy(), parents[k][1], value, b])
        else:
            n_keep = 3
            moves = moves_without_blunder(b, eval_board, moves, side, n_keep, init==False)
            for i in range(len(moves)):
                temp.append([moves[i][0], parents[k][1]+ [i], moves[i][1], 0 ,0])
            if side:
                value = max(temp, key=lambda x: x[2])
            else:
                value = min(temp, key=lambda x: x[2])
            children.append([temp.copy(), parents[k][1], value[2], value[0]])
    return(children)
def moves_without_blunder(b, eval_board, moves, side, n_keep, blunder):
    l = len(moves)
    values = []
    if (l <= n_keep) or (blunder):
        for k in range(l):
            copy = b.copy()
            copy.push(moves[k])
            v = eval_board(copy, 0)
            values.append([copy, v]) 
        return(values)
    else:
        initial = eval_board(b, 0)
        for k in range(l):
            copy = b.copy()
            copy.push(moves[k])
            v = eval_board(copy, 5)
            keep = 0
            if (v-initial)*(1 if side else -1) <-1:
                keep = -1
            if (v-initial)*(1 if side else -1) > 1:
                keep = 1
            values.append([copy, v, keep])    
        normal = [e[:2] for e in values if e[2]==0]
        best = [e[:2] for e in values if e[2]==1]
        worst = [e[:2] for e in values if e[2]==-1]
        if best !=[]:
            best.sort(key=lambda l: l[1], reverse=side)
            return(best[:1])
        else:
            #if best !=[]:
                
                #if len(best)+len(normal) >=n_keep:
                    #normal.sort(key=lambda l: l[1], reverse=side)
                    #return(best + normal[:n_keep-len(best)])
                #else:
                    #worst.sort(key=lambda l: l[1], reverse=side)
                    #return(best + normal+ worst[:n_keep-len(best)-len(normal)])
            
            if len(normal) >=n_keep:
                return(normal)
            else:
                worst.sort(key=lambda l: l[1], reverse=side)
                return(normal+ worst[:n_keep-len(normal)])
                
                

In [13]:
#hunt
#children = [[[list of children], position, value], [[list of children], position, value]...]
#grouped = [[[[list of children], position, value]], [[list of children], position, value]], [[[list of children], position, value]]...]
def growing(children, parameters, eval_board, side):
    grouped = group_with_relatives(children, len(children[0][1])-2)
    children_dict = {}
    for k in range(len(grouped)):
        if side:
            v = min(grouped[k], key=lambda x: x[2])
            updated_value = min(eval_board(v[3], parameters.depth-1), v[2])
        else:
            v = max(grouped[k], key=lambda x: x[2])
            updated_value = max(eval_board(v[3], parameters.depth-1), v[2])
        boards = [l[0] for l in grouped[k]]
        position = grouped[k][0][1][:-1]
        grouped[k] = [[len(boards)], position, updated_value, 1]
        children_dict[str(position)] = boards
    return(grouped, children_dict)
def sort_hunters(groups, depth, side):
    if depth<0:
        return(groups)
    else:
        for k in range(len(groups)):
            groups[k] = [0, groups[k][0][1][:-1], groups[k]]
        groups = group_with_relatives(groups, depth)
        #groups = [family,family,...]
        #family = [[0, position, sorted_list], [0, position, sorted_list],[0, position, sorted_list]...]
        #sorted_list = [[board, position, score...], [board, position, score...], [board, position, score...]]
        for k in range(len(groups)): #pour chaque famille
            family = groups[k]
            new = []
            ind = 0 #On commence par prendre le meilleur
            p = True
            while p:
                l = []
                for i in range(len(family)): 
                    if ind < len(family[i][2]):
                        l.append(family[i][2][ind])
                if l!=[]:
                    l.sort(key=lambda l: l[2], reverse=side)
                    new +=l
                    ind += 1
                else:   
                    p = False   
            groups[k] = new
        return(sort_hunters(groups, depth-1, side==False))
def score_hunters(hunters, tree_depth, parameters):
    contrast = parameters.contrast[tree_depth]
    deplete_rate = parameters.deplete_rate[tree_depth]
    width  = parameters.width[tree_depth]
    l = len(hunters)
    if l==0:
        return([])
    elif l ==1:
        hunters[0][3] = width
        return(hunters)
    elif l==2:
        rate = 1/contrast
    else:    
        rate = 1/contrast**(1/(l-1))
    weights = [rate ** (i**deplete_rate*(l-1)**(1-deplete_rate)) for i in range(l)]
    norm = width/sum(weights)
    for k in range(l):
        hunters[k][3] = norm*weights[k]
    return(hunters)
def hunt(hunters, tree_depth, parameters, side): 
    #tree_depth = i+1 = len(position)
    for k in range(len(hunters)):
        hunters[k] = [hunters[k]]
    sorted_hunters = sort_hunters(hunters, tree_depth-3, side)[0]
    print("scoring")
    scored_hunters = score_hunters(sorted_hunters, tree_depth, parameters) 
    approxed_hunters = approx(scored_hunters, parameters.minimal[tree_depth-1])
    a = [sum(e[3]) for e in approxed_hunters if sum(e[3])>0.3]
    print(a)
    print(sum(a))
    return(approxed_hunters)

def approx(hunters, minimal):
    keep = []
    for k in range(len(hunters)):
        f = hunters[k][3]
        n_ = hunters[k][0][0]
        nc = f/n_
        if nc <1:
            nc = np.random.binomial(1,int(nc*100)/100)
            if nc ==0:
                continue
            else:
                N = [1]*n_
        else:
            N = [nc]*n_
        for ind in range(len(N)):
            i = np.floor(N[ind]/minimal)*minimal
            p = int((N[ind]-i)*100)/minimal/100
            N[ind] = int(i + minimal*np.random.binomial(1,p))
        new = [n for n in N if n>0]
        if len(new) >= 1:
            hunters[k][3] = N
            keep.append(k)
    hunters = [hunters[i] for i in keep] 
    return(hunters)

def group_with_relatives(L, depth):
    result = {}
    for l in L:
        if len(l)<=1:
            print(l)
        prefix = l[1][:depth+1]
        if str(prefix) in result:
            result[str(prefix)].append(l)
        else:
            result[str(prefix)] = [l]
    return (list(result.values()))
def flatten_list(lst):
    flattened_list = []
    for item in lst:
        if isinstance(item[0], list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

In [14]:
#children selection
def reasign(children, dict_children):
    for k in range(len(children)):
        children[k][0] = dict_children[str(children[k][1])]
    return(children)
def gen_N(n, n_):
    q = n//n_
    r = n%n_
    random_integers = random.sample(range(n_), r)
    N = [q]*n_
    for i in range(r):
        N[random_integers[i]] +=1
    return(N)
def select(children, side, eval_board, I, parameters):
    survived = []
    killed = 0
    for k in range(len(children)):
        N = children[k][3]
        n_ = len(children[k][0])
        for j in range(n_):
            s, kill = keep(children[k][0][j], N[j], eval_board, I, parameters, side)
            if I >= parameters.depth-1:
                for i in range(len(s)):
                    if s[i][3] == 0:
                        s[i][2] = eval_board(s[i][0], parameters.depth-1)          
            survived += s
            killed +=kill
    print("survived: " + str(len(survived)))
    print("killed: " + str(killed))
    return(survived)
def keep(L, n, eval_board, I, parameters, side):
    if n==0:
        return([], len(L))
    depth = len(L[0][1])-1
    if len(L)<= n:
        return L, 0
    else:
        L.sort(key=lambda l: l[2], reverse=side)
        w2 = int(np.ceil((1-parameters.risk[depth])*n))
        if I >= parameters.depth-1:
            for k in range(w2):
                b = L[k][0]
                move = b.peek()
                if (b.is_attacked_by(b.turn, move.from_square) or b.is_capture(move)):
                    L[k][2] = eval_board(b, parameters.depth-1)
                    L[k][3] = 1  
        p = L[:w2]
        rest = L[w2:]
        l = len(rest)
        if l <=1:
            return L[:n], len(L)-n
        rate = 1/parameters.contrast[depth]**(1/(l-1))
        weights = [rate ** i for i in range(l)]
        s = unique(random.choices(rest, weights, k=n - w2))
        survived = p + s
        return survived, len(L)-len(survived)
def unique(L):
    unique_dict = {}
    for sublist in L:
        value = sublist[0].fen()
        if value not in unique_dict:
            unique_dict[value] = sublist
    unique_list = list(unique_dict.values())
    return unique_list


In [15]:
#up propagation
def keep_best(children, side):
    depth = len(children[0][1])
    groups = group_with_relatives(children, depth-2)
    best = []
    for group in groups:
        if side:
            best.append(max(group, key=lambda x: x[3]))
        else:
            best.append(min(group, key=lambda x: x[3]))
    return(best)
"""def clean_tree(tree):
    depth = len(tree)
    for k in range(depth-2):
        parent = 
        children = {str(l2[1][:-1]): l2[2] for l2 in tree[depth-k-1]}
        for parent in tree[depth-k-2]:
            if str()"""
    
def up_propagate(tree, side, l):
    if len(tree)<=2:
        L2 = keep_best(tree[-1], side)
        l.append(L2)
        b = L2[0][0].copy()
        return b.pop(), l
    else:
        L2 = keep_best(tree[-1], side)          
        l.append(L2)
        dict_l2 = {str(l2[1][:-1]): l2[3] for l2 in L2}
        new = tree[-2].copy()
        for l1 in new:
            if str(l1[1]) in dict_l2:
                l1[3] = dict_l2[str(l1[1])]
            else : 
                l1[3] = 9999
        tree[-2] = [n for n in new if n[3] != 9999]
        return(up_propagate(tree[:-1], side==False, l))

def print_steps(l):
    l.reverse()
    next_ = [0]
    for k in range(len(l)):
        P = True
        for e in l[k]:
            if e[1][:-1]==next_:
                next_ = e[1]
                print(e[0])
                print("value = " + str(e[2]))
                print("updated value = " + str(e[3]))
                P = False
                continue
        if P:
            print("not found " + str(next_))

In [16]:
#recycle
def recycle(l ,b ,side):
    pos = []
    if l ==[]:
        return(False, 0)
    for e in l[0]:
        if e[0]==b:
            pos = e[1]
    if pos ==[]:
        return(False, 0)
    else:
        L = []
        for e_ in l[1]:
            if e_[1][:-1]==pos:
                L.append(e_)
        L.sort(key=lambda l: l[3], reverse=side)
        print(L)
        return(True, L[0][0].pop())
    
           

In [17]:
#parameters
class Parameters:
    def __init__(self, depth, width, risk, contrast, deplete_rate, minimal, agressivity):
        self.depth = depth
        self.width = width
        self.risk = risk
        self.contrast = contrast
        self.deplete_rate = deplete_rate
        self.minimal = minimal
        self.agressivity = agressivity
    def random_modify(self):
        self.width = [w * random.uniform(0.9, 1.1) for w in self.width]
        self.risk = [r * random.uniform(0.9, 1.1) for r in self.risk]
        self.contrast = [c * random.uniform(0.9, 1.1) for c in self.contrast]
        self.aggressivity *= random.uniform(0.9, 1.1)

In [18]:
agent_color = True
def play_min_maxN(tree, subtree):
    depth = 4
    #max depth = depth + 3  
    width = [1, "?", 100, 200, 300, 400, 600]
    risk = ["?","?",0.6,0.5,0.4,0.25, 0.2] # risk[0] selection pour les enfants de la racine
    contrast = ["?","?",7,10,12,13,14] #contrast[1] contraste pour la sélection et distribution de ressource pour la couche 1 
    deplete_rate = ["?","?",1,0.9,0.8,0.7,0.6]
    minimal = ["?","?",2,2,3,4,5]
    agressivity = 0.5
    parameters = Parameters(depth, width, risk, contrast, deplete_rate, minimal, agressivity)
    color = agent_color
    return darwinian_tree(tree, subtree, parameters, stable_eval, color)

In [19]:
b = chess.Board()
main_one_agent(b,play_min_maxN,agent_color)

time used: 0.0038437843322753906 seconds
time used: 0.003350496292114258 seconds
time used: 0.0032994747161865234 seconds
-------------
initial
initial board value = 0.87
game_stage = [0.84, 0.12, 0.0, 0.01]
r n b q k b n r
. p . p p p . p
. . . . . . . .
p . p P . . p .
. . . . P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
time used: 0.09919142723083496 seconds
-------------
initial
initial board value = 2.268157894736842
game_stage = [0.88, 0.09, 0.0, 0.01]
r n b q k b n r
. . . p p p . p
. . . . . . . .
p p p P . . B .
. . . . P . . .
. . . . . . . .
P P P . . P P P
R N . Q K B N R
time used: 0.276456356048584 seconds
-------------
initial
initial board value = 3.446756756756757
game_stage = [0.88, 0.09, 0.0, 0.01]
r n b q k b n r
. . . p . p . p
. . . . . . . .
p B p P p . B .
. . . . P . . .
. . . . . . . .
P P P . . P P P
R N . Q K . N R
time used: 0.9368901252746582 seconds
-------------
initial
initial board value = 11.30142857142857
game_stage = [0.11, 0.85, 0.01, 0.

growing
hunting
scoring
[32, 30, 20, 22, 18, 14, 12, 12, 12, 6, 8, 6, 2, 8, 8]
210
selecting
survived: 209
killed: 3678
exploration 75.0 %
reproducing
growing
hunting
scoring
[14, 12, 14, 12, 10, 12, 12, 8, 10, 10, 10, 8, 8, 6, 6, 8, 8, 4, 4, 8, 6, 4, 6, 6, 6, 4, 4, 4, 4, 2, 4, 4, 4, 2, 2, 8, 4, 4, 4, 6, 2, 4, 4, 4, 4, 6, 6, 4, 6, 6, 4, 4, 2]
328
selecting
survived: 307
killed: 1084
exploration 100.0 %
[1, 16, 80, 209, 307]
up propagating
showing steps
. . b . . . . k
. . . . Q . . .
. . . . . . . .
. . . P . . . P
. . p . P . . .
. . N . . N . .
P P P . . P P .
. . K R . . . R
value = 255.3375
updated value = 1063.26
. . . . . . . k
. . . . Q . . .
. . . . b . . .
. . . P . . . P
. . p . P . . .
. . N . . N . .
P P P . . P P .
. . K R . . . R
value = 252.6975
updated value = 1063.26
. . . . . . . k
. . . . . . . .
. . . . Q . . .
. . . P . . . P
. . p . P . . .
. . N . . N . .
P P P . . P P .
. . K R . . . R
value = 1088.67
updated value = 1063.26
. . . . . . . .
. . . . . . k .
. . .

[[Board('3Q3k/8/7P/1R6/P1p1PBP1/2N2N2/1PP2P2/1K1R4 b - - 12 42'), [0, 0, 0, 2], 1299.5500000000002, 1298.5, 0]]
recycle
time used: 0.0065991878509521484 seconds
-------------
initial
initial board value = 1298.5
game_stage = [0.0, 0.77, 0.17, 0.04]
. . . Q . . . .
. . . . . . . k
. . . . . . . P
. R . . . . . .
P . p . P B P .
. . N . . N . .
. P P . . P . .
. K . R . . . .
1st
width[1] = 3
width[2] = 5
start exploration
reproducing
growing
hunting
scoring
[144, 40, 16]
200
selecting
survived: 175
killed: 178
exploration 75.0 %
reproducing
growing
hunting
scoring
[158, 72, 26, 38]
294
selecting
survived: 141
killed: 99
exploration 100.0 %
[1, 3, 5, 175, 141]
up propagating
showing steps
. . . . . . . .
. . . . . . . k
. . . . . Q . P
. R . . . . . .
P . p . P B P .
. . N . . N . .
. P P . . P . .
. K . R . . . .
value = 1297.45
updated value = 1299.9
. . . . . . k .
. . . . . . . .
. . . . . Q . P
. R . . . . . .
P . p . P B P .
. . N . . N . .
. P P . . P . .
. K . R . . . .
value = 1

-------------
initial
initial board value = 1498.87
game_stage = [0.0, 0.84, 0.1, 0.04]
. . . . . . . R
. . B . . . . .
. . . . Q . . .
. R . . . . P .
P . p B P . . .
. . N . . N . .
. P P . . P k .
. K . . . . . .
1st
width[1] = 3
width[2] = 4
start exploration
reproducing
growing
hunting
scoring
[142, 42, 16]
200
selecting
survived: 164
killed: 137
exploration 75.0 %
reproducing
growing
hunting
scoring
[194, 68, 28]
290
selecting
survived: 119
killed: 6
exploration 100.0 %
[1, 3, 4, 164, 119]
up propagating
showing steps
. . . . . R . .
. . B . . . . .
. . . . Q . . .
. R . . . . P .
P . p B P . . .
. . N . . N . .
. P P . . P k .
. K . . . . . .
value = 1495.54
updated value = 1498.87
. . . . . R . .
. . B . . . . .
. . . . Q . . .
. R . . . . P .
P . p B P . . .
. . N . . N . .
. P P . . P . .
. K . . . . . k
value = 1534.3899999999999
updated value = 1498.87
. . . . . . . R
. . B . . . . .
. . . . Q . . .
. R . . . . P .
P . p B P . . .
. . N . . N . .
. P P . . P . .
. K . . . .

In [20]:
"""def group_hunters(hunters, depth, max_depth, width, contrast, side):
    if depth > max_depth:
        return(hunters)
    else:
        groups = group_with_relatives(hunters, depth)
        for k in range(len(groups)):
            group = groups[k]
            if width <0:
                width = 1
                #print(width)
            else:
                width = 0
                for g in group:
                    width +=g[4]
            if len(group) <=2:
                for i in range(len(group)):
                    group[i][4] = width/len(group)
                ranked_hunters = group
            else:
                ranked_hunters = rank_hunters(group, width, contrast*np.log(len(group))**4*0.2, side)
            #print(ranked_hunters)
            groups[k] = group_hunters(ranked_hunters, depth+1, max_depth, width, contrast, side==False)
    return(groups)
def rank_hunters(hunters, width, contrast, side):
    hunters.sort(key=lambda l: l[2], reverse=side)
    l = len(hunters)
    S = 0
    for k in range(l):
        s = hunters[k][4]*contrast**((l-k)/l)
        hunters[k][4] = s
        S += s
    norm = width/S
    for k in range(l):
        hunters[k][4] *= norm
    return(hunters)
def resize(hunters, contrast, width):
    m = min(hunters, key=lambda x: x[4])
    M = max(hunters, key=lambda x: x[4])
    for k in range(len(hunters)):
        hunters[k] = (hunters[k]-m)/(M-m)*(contrast-1) + 1
    S = 0
    for k in range(len(hunters)):
        S += hunters[4]
    for k in range(len(hunters)):
        hunters[k] = hunters[k]/S*width
    return(hunters)"""


'def group_hunters(hunters, depth, max_depth, width, contrast, side):\n    if depth > max_depth:\n        return(hunters)\n    else:\n        groups = group_with_relatives(hunters, depth)\n        for k in range(len(groups)):\n            group = groups[k]\n            if width <0:\n                width = 1\n                #print(width)\n            else:\n                width = 0\n                for g in group:\n                    width +=g[4]\n            if len(group) <=2:\n                for i in range(len(group)):\n                    group[i][4] = width/len(group)\n                ranked_hunters = group\n            else:\n                ranked_hunters = rank_hunters(group, width, contrast*np.log(len(group))**4*0.2, side)\n            #print(ranked_hunters)\n            groups[k] = group_hunters(ranked_hunters, depth+1, max_depth, width, contrast, side==False)\n    return(groups)\ndef rank_hunters(hunters, width, contrast, side):\n    hunters.sort(key=lambda l: l[2], rever